In [3]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree, add_self_loops
from torch_scatter import scatter

/home/kris/anaconda3/envs/GNN/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<!-- The GCN layer is defined as:
$$ x_{i}^{k} = \sum_{j \in N(i) \cup {i}} \dfrac{1}{\sqrt{deg(i)} \sqrt{deg(j)}} (W^{T} x_{j}^{k-1} + b)$$

Step:  
1.Add self-loops   
2.Linear transform node feature matrix(y = Wx)  
3.Compute coefficients (the deg part)  
4.Normalize passing node features in message function  
5.Sum up neighboring node features ("add")  
6.Add bias vector

 -->
 Customized aggregate function so than inputs value < 1 are ignored

In [25]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__() #step 5
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))
        
        self.reset_parameters()
        
    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()
        
    def forward(self, x, edge_index):
        #step 1
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        #step 2
        x = self.lin(x)
        #step 3
        inpoints, outpoints = edge_index[0], edge_index[1]
        deg = degree(inpoints, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[inpoints] * deg_inv_sqrt[outpoints]
        #step 4 and 5
        out = self.propagate(edge_index, x=x, norm=norm)
        #step 6
        out += self.bias
        
        return out
    
    def message(self, x_j, norm):
        return norm.view(-1,1) * x_j

In [26]:
class GCNConvnew(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__() #step 5
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))
        
        self.reset_parameters()
        
    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()
        
    def forward(self, x, edge_index):
        #step 1
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        #step 2
        x = self.lin(x)
        #step 3
        inpoints, outpoints = edge_index[0], edge_index[1]
        deg = degree(inpoints, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[inpoints] * deg_inv_sqrt[outpoints]
        #step 4 and 5
        out = self.propagate(edge_index, x=x, norm=norm)
        #step 6
        out += self.bias
        
        return out
    
    def message(self, x_j, norm):
        return norm.view(-1,1) * x_j
    def aggregate(self, inputs, index):
        inputs[inputs==torch.clamp(inputs, -1, 1)] = 0
        return scatter(inputs, index, dim=0, reduce="sum")

In [27]:
conv = GCNConv(2, 4)
convnew = GCNConvnew(2, 4)
x = torch.rand(4,2) * 20
edge_index = torch.tensor([[0, 1, 1, 2, 0, 2, 0, 3],
                           [1, 0, 2, 1, 2, 0, 3, 0]])
x_out = conv(x, edge_index)
print(x_out.shape)
print(x_out)
print("----------GCN with new aggregate function-------------")
x_out = convnew(x, edge_index)
print(x_out.shape)
print(x_out)

torch.Size([4, 4])
tensor([[ 2.1104,  8.8012, 10.7248, 11.6071],
        [ 2.9415,  8.7290, 10.2896, 10.9208],
        [ 2.9415,  8.7290, 10.2896, 10.9208],
        [ 0.7667,  4.0976,  5.0815,  5.5544]], grad_fn=<AddBackward0>)
----------GCN with new aggregate function-------------
torch.Size([4, 4])
tensor([[ 3.1652,  5.6372, -8.7327, 11.1287],
        [ 3.6548,  6.5093, -8.2487, 10.3107],
        [ 3.6548,  6.5093, -8.2487, 10.3107],
        [ 1.8318,  2.0715, -4.1706,  5.3661]], grad_fn=<AddBackward0>)
